In [1]:
!pip install sentence-transformers --quiet


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np

c:\Users\USER\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("Loading embedding model...")
model = SentenceTransformer("all-MiniLM-L6-v2")
print(f"Model produces {model.get_sentence_embedding_dimension()} dimensional embeddings")

'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000130679EE120>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: ba5c1a1a-62eb-4a01-8dd6-9f7378252820)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


Loading embedding model...


'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001306793FC50>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 6b705c5e-e1ed-4b5c-b04b-46cbe1fa4c60)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 2s [Retry 2/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000013067A602D0>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 27c29660-0d20-4bf3-866b-498784e547a6)')' thrown while requesting HEAD https://huggingface.

Model produces 384 dimensional embeddings


In [3]:
text = "The cat sat on the mat"
embedding = model.encode(text)
print(f"Original text {text}")
print(f"Embedding shape: {embedding.shape}")
print(f"First ten values: {embedding[:10]}")

Original text The cat sat on the mat
Embedding shape: (384,)
First ten values: [ 0.13040183 -0.01187012 -0.02811707  0.05123867 -0.05597445  0.03019156
  0.03016133  0.02469836 -0.01837058  0.0587668 ]


In [4]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    vec1_scalar = np.linalg.norm(vec1)
    vec2_scalar = np.linalg.norm(vec2)
    return dot_product / (vec1_scalar * vec2_scalar)

In [5]:
sentences = [
    "The cat sat on the mat",
    "A feline rested on the rug",      # Similar meaning, different words
    "Dogs are loyal animals",          # Different topic
    "Python is a programming language" # Completely unrelated
]

embeddings = model.encode(sentences)
embeddings.shape

(4, 384)

In [6]:
for i, sentence in enumerate(sentences):
    similarity = cosine_similarity(embeddings[0], embeddings[i])
    print(f"Similarity to {sentence}")
    print(f"Score: {similarity}\n")

Similarity to The cat sat on the mat
Score: 1.0000001192092896

Similarity to A feline rested on the rug
Score: 0.5643377304077148

Similarity to Dogs are loyal animals
Score: 0.16523928940296173

Similarity to Python is a programming language
Score: 0.0308724083006382



### Simple Semantic Search

In [7]:
# Sample knowledge base
documents = [
    "Python is a high-level programming language known for simplicity",
    "Machine learning enables computers to learn from data",
    "Neural networks are inspired by biological brains",
    "Dogs are loyal and friendly pets that need exercise",
    "Cats are independent animals that make great companions",
    "JavaScript is used for web development and runs in browsers",
    "Deep learning uses multi-layered neural networks",
    "Puppies require training and socialization from an early age"
]

print(f"Knowledge base: {len(documents)} documents")

Knowledge base: 8 documents


In [8]:
#Create embeddings
embeddings = model.encode(documents)

print(f"Created {len(embeddings)} embeddings")
print(f"Each embedding has {embeddings[0].shape[0]} dimensions")

Created 8 embeddings
Each embedding has 384 dimensions


In [9]:
def search(query, documents, doc_embeddings, top_k):
    query_embedding = model.encode(query)
    similarities = []
    for index, embedding in enumerate(doc_embeddings):
        similarity = cosine_similarity(embedding, query_embedding)
        similarities.append((documents[index], similarity))

    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_k]



In [10]:
# Test different queries
queries = [
    "What is artificial intelligence?",
    "Tell me about pet dogs",
    "How do I code in Python?"
]

for query in queries:
    print(f"\n{'='*80}")
    print(f"QUERY: {query}")
    print(f"{'='*80}")
    
    results = search(query, documents, embeddings, top_k=3)
    
    for i, (doc, score) in enumerate(results, 1):
        print(f"\n{i}. (Score: {score:.3f})")
        print(f"   {doc}")


QUERY: What is artificial intelligence?

1. (Score: 0.408)
   Machine learning enables computers to learn from data

2. (Score: 0.395)
   Neural networks are inspired by biological brains

3. (Score: 0.326)
   Python is a high-level programming language known for simplicity

QUERY: Tell me about pet dogs

1. (Score: 0.548)
   Dogs are loyal and friendly pets that need exercise

2. (Score: 0.437)
   Puppies require training and socialization from an early age

3. (Score: 0.413)
   Cats are independent animals that make great companions

QUERY: How do I code in Python?

1. (Score: 0.554)
   Python is a high-level programming language known for simplicity

2. (Score: 0.148)
   Puppies require training and socialization from an early age

3. (Score: 0.138)
   JavaScript is used for web development and runs in browsers


In [11]:
results

[('Python is a high-level programming language known for simplicity',
  np.float32(0.5542636)),
 ('Puppies require training and socialization from an early age',
  np.float32(0.14795657)),
 ('JavaScript is used for web development and runs in browsers',
  np.float32(0.13812675))]

In [12]:
# Load two different models for comparison
print("Loading models...\n")

model_small = SentenceTransformer('all-MiniLM-L6-v2')      # 384 dimensions
model_large = SentenceTransformer('all-mpnet-base-v2')     # 768 dimensions

print("✅ Both models loaded!")
print(f"Small model: {model_small.get_sentence_embedding_dimension()} dimensions")
print(f"Large model: {model_large.get_sentence_embedding_dimension()} dimensions")

'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000013067A639D0>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: f691ae1b-0317-40d7-a55f-7b4308a1ed90)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


Loading models...



'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000013067A63B10>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 36594ff4-341f-4adb-9253-267db79fedc0)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 2s [Retry 2/5].
'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000013067A63750>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: 3d8ebdb0-6563-4cf3-a808-942de0ad4a50)')' thrown while requesting HEAD https://huggingface.

✅ Both models loaded!
Small model: 384 dimensions
Large model: 768 dimensions


In [16]:
# Compare on a similarity task
test_pairs = [
    ("The dog is running", "A canine is jogging"),           # Similar
    ("I love pizza", "Pizza is delicious"),                  # Related
    ("Python programming", "Cooking pasta")                  # Unrelated
]

print("Comparing model performance:\n")
for text1, text2 in test_pairs:
    # Small model
    emb1_small = model_small.encode([text1, text2])
    sim_small = cosine_similarity(emb1_small[0], emb1_small[1])
    
    # Large model  
    emb1_large = model_large.encode([text1, text2])
    sim_large = cosine_similarity(emb1_large[0], emb1_large[1])
    
    print(f"Pair: '{text1}' vs '{text2}'")
    print(f"  Small model: {sim_small:.3f}")
    print(f"  Large model: {sim_large:.3f}")
    print()

Comparing model performance:

Pair: 'The dog is running' vs 'A canine is jogging'
  Small model: 0.818
  Large model: 0.827

Pair: 'I love pizza' vs 'Pizza is delicious'
  Small model: 0.801
  Large model: 0.785

Pair: 'Python programming' vs 'Cooking pasta'
  Small model: 0.142
  Large model: 0.120



In [20]:
class SimpleRetriever:
    def __init__(self, model_name = "all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)
        self.chunks = []
        self.embeddings = None
    
    #Add documents and chunk
    def add_documents(self, documents, chunk_size = 500):
        """
        Add documents to the retriever(and chunk them)
        """

        #Simple Chunking
        for doc in documents:
            words = doc.split()
            for i in range(0, len(words), chunk_size):
                chunk = ' '.join(words[i:i+chunk_size])
                self.chunks.append(chunk)

        #Generate embeddings
        print(f"Embedding {len(self.chunks)} chunks...")
        self.embeddings = self.model.encode(self.chunks)
        print(f"✅ Ready! {len(self.chunks)} chunks indexed.")
        print(self.chunks[:2])
    
    def search(self, query, top_k=3):
        """
        Search for relevant chunks.
        """
        # Embed query
        query_embedding = self.model.encode(query)
        
        # Calculate similarities
        similarities = []
        for i, chunk_emb in enumerate(self.embeddings):
            sim = cosine_similarity(query_embedding, chunk_emb)
            similarities.append((self.chunks[i], sim))
        
        # Sort and return top k
        similarities.sort(key=lambda x: x[1], reverse=True)
        return similarities[:top_k]

print("✅ SimpleRetriever class ready!")


✅ SimpleRetriever class ready!


In [21]:
# Test it with sample documents
sample_docs = [
    """
    Python is a versatile programming language widely used in web development,
    data science, and automation. Its simple syntax makes it beginner-friendly
    while remaining powerful for advanced applications.
    """,
    """
    Machine learning is a subset of artificial intelligence that enables systems
    to learn and improve from experience. Popular frameworks include TensorFlow,
    PyTorch, and scikit-learn.
    """,
    """
    Dogs are loyal companions that require regular exercise, training, and
    veterinary care. Different breeds have varying needs and temperaments.
    """
]

# Create retriever and add documents
retriever = SimpleRetriever()
retriever.add_documents(sample_docs, chunk_size=100)



Embedding 3 chunks...
✅ Ready! 3 chunks indexed.
['Python is a versatile programming language widely used in web development, data science, and automation. Its simple syntax makes it beginner-friendly while remaining powerful for advanced applications.', 'Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience. Popular frameworks include TensorFlow, PyTorch, and scikit-learn.']


In [19]:
check = retriever.add_documents(sample_docs, chunk_size=100)
check

Embedding 6 chunks...
✅ Ready! 6 chunks indexed.


In [ ]:
|